# MCP

In [14]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace, WebSearchTool
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [3]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate EXA Search as an MCP:

In [8]:
# Exa Search MCP code comes here:
exa_search_mcp = MCPServerStreamableHttp(
    name ="Exa Search MCP",
    params={
        "url": f"https://mcp.exa.ai/mcp?exaApiKey={os.environ.get('EXA_API_KEY')}",
        "timeout": 30
    },
    client_session_timeout_seconds=30,
    cache_tools_list=True,
    max_retry_attempts=1
)

await exa_search_mcp.connect()

calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool],
    mcp_servers=[exa_search_mcp]
)

Reference query - shouldn't use ExaSearch:

In [9]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Banana (1 medium, ~118 g): ~105 kcal
    - Apple (1 medium, ~182 g): ~95 kcal
    - Total: ~200 kcal
    
    Calories per 100 g:
    - Banana: 89 kcal/100 g
    - Apple: 52 kcal/100 g
    
    Note: actual calories vary with fruit size.
- 7 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [10]:
with trace("Nutrition Assistant with MCP "):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A standard full English breakfast (1 serving) contains about 935 kcal, depending on exact ingredients/portions.

Ingredients per serving and approximate calories:
- Pork sausage: 1 medium (60 g) ≈ 148 kcal
- Bacon: 1 strip (30 g) ≈ 122 kcal
- Eggs: 2 medium (200 g total) ≈ 194 kcal
- Butter: 1 tablespoon (30 g) ≈ 215 kcal
- Olive oil: 2 teaspoons (10 ml) ≈ 79 kcal
- Baked beans: 1/2 cup (100 g) ≈ 94 kcal
- Bread (toast): 1 slice (35 g) ≈ 83 kcal

Total ≈ 935 kcal per serving. Note: exact calories vary by brand/size.


In [15]:
calorie_agent_with_websearch = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool, WebSearchTool()]
)

In [16]:
with trace("Nutrition Assistant with OpenAI Web Search "):
    result = await Runner.run(
        calorie_agent_with_websearch, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A traditional full English breakfast is roughly 800–1,000 calories per serving. Exact calories vary a lot by portions and ingredients.

Sample (typical portions):
- 2 sausages: ~180–250 kcal each (360–500)
- 2 rashers bacon: ~90–120 kcal each (180–240)
- 2 eggs: ~70–90 kcal each (140–180)
- Baked beans (1/2 cup): ~120–130
- Toast (2 slices): ~120–160
- Mushrooms (sautéed, 1/2 cup): ~30–50
- Grilled tomato (1): ~5–15
- Hash browns (1 small): ~120–150
Total: ~1,000–1,200 kcal

If you tell me the exact ingredients and portions you’re using, I’ll calculate a precise total.
